In [10]:
import pandas as pd
import openpyxl, os, re

In [11]:
input_file = '/Users/andy/Desktop/20191120.3chemo/17A030864.chemo.v2.txt'


In [12]:
def correctGT(input_file, dic_name):
    data = pd.read_csv(input_file, sep='\t')
    list_correctGT = []
    reverse_complement = lambda gt: gt.replace('A', 't').replace('T', 'a').replace('G', 'c').replace('C', 'g').upper()[::-1]
    complement = lambda gt: gt.replace('A','t').replace('T','a').replace('G','c').replace('C','g').upper()
    for key in dic_name.keys():
        if data.loc[data['RSID']==key, 'Genotype'].empty == True:
            list_correctGT.append(key + '--' + '基因型不存在')
        else:
            if data.loc[data['RSID']==key, 'Genotype'].all() in dic_name[key]:
                list_correctGT.append(key + '--' + data.loc[data['RSID']==key, 'Genotype'].all())
            elif complement(data.loc[data['RSID']==key, 'Genotype'].all()) in dic_name[key]:
                list_correctGT.append(key + '--' + complement(data.loc[data['RSID']==key, 'Genotype'].all()))
            elif reverse_complement(data.loc[data['RSID']==key, 'Genotype'].all()) in dic_name[key]:
                list_correctGT.append(key + '--' + reverse_complement(data.loc[data['RSID']==key, 'Genotype'].all()))
            elif data.loc[data['RSID']==key, 'Genotype'].all()[::-1] in dic_name[key]:
                list_correctGT.append(key + '--' + data.loc[data['RSID']==key, 'Genotype'].all()[::-1])
            elif '6' or '7' in data.loc[data['RSID']==key, 'Genotype'].all():
                if data.loc[data['RSID']==key, 'Genotype'].all().count('6') == 1:
                    list_correctGT.append(key + '--' + '(TA)6/(TA)7')
                elif data.loc[data['RSID']==key, 'Genotype'].all().count('6') == 2:
                    list_correctGT.append(key + '--' + '(TA)6/(TA)6')
                elif data.loc[data['RSID']==key, 'Genotype'].all().count('6') == 0:
                    list_correctGT.append(key + '--' + '(TA)7/(TA)7')
    return list_correctGT


In [13]:
result_50gene = '/Users/andy/Desktop/化疗药/泛癌种50基因/泛癌50基因20191015.xlsx' #各癌种自动化流程输入文件
project_file = '/Users/andy/Desktop/临床样本项目信息11.24.xlsx' #项目信息文件表
mypath = '/Users/andy/Desktop/20191127_ST1122.chemo' #提取基因型文件存放目录
project_dic = {} #提取基因型文件对应所做项目及客户姓名映射字典
for i in os.listdir(mypath):
    info_data = pd.read_excel(project_file)
    if i != '.DS_Store':
        project_dic[info_data.loc[info_data['样本编号']==i.split('.')[0], '患者姓名'].all()] = info_data.loc[info_data['样本编号']==i.split('.')[0], '检测项目'].all() + '--' + i

project_dic
#os.listdir(mypath)

{'谭兴英': '肺癌37基因（靶向+化疗+癌痛）--17A010764.spark.chemo.v2.txt',
 '刘淑玲': '肺癌37基因（靶向+化疗+癌痛）--17A030923.spark.chemo.v2.txt',
 '郭元海': '实体瘤泛癌种50基因（靶向+化疗+癌痛）--17A030920.spark.chemo.v2.txt',
 '曹喜成': '肺癌37基因（靶向+化疗+癌痛）--17A030918.spark.chemo.v2.txt'}

In [14]:
gene_37 = {}
gene_50 = {}
gene_500 = {}
gene_aitong = {}
gene_aitong500 = {}
###################
dic_37gene = {'rs7311358':['AA', 'GA', 'GG'], 'rs1127687':['AA', 'GA', 'GG'], 'rs4353229':['TT', 'TC', 'CC'], 'rs12806698':['CC', 'AA', 'CA'],
'rs1801133':['AA', 'GA', 'GG'], 'rs716274':['AA', 'AG', 'GG'], 'rs11615':['CC', 'CT', 'TT'], 'rs1695':['AA', 'AG', 'GG'], 'rs2072671':['AA', 'AC', 'CC'], 'rs11545078':['CC', 'CT', 'TT'], 'rs1517114':['GG', 'CG', 'CC'], 'rs8175347':['(TA)6/(TA)6', '(TA)6/(TA)7', '(TA)7/(TA)7']}

dic_50gene = {'rs25487':['AA', 'AG', 'GG'], 'rs1801133':['AA', 'GA', 'GG'], 'rs11615':['TT', 'CT', 'CC'], 'rs1695':['AA', 'AG', 'GG'], 'rs67376798':['AA', 'TA', 'TT'], 'rs1801019':['GG', 'GC', 'CC'], 'rs1042522':['GG', 'CG','CC'], 'rs4148323':['AA', 'GA', 'GG'], 
'rs1517114':['GG', 'CG', 'CC'], 'rs1056892':['AA', 'GA', 'GG'], 'rs3892097':['TT','CT', 'CC'], 'rs3918290':['TT', 'CT', 'CC'], 
'rs9937':['AA', 'AG', 'GG'], 'rs11598702':['TT', 'TC', 'CC'], 'rs2228001':['TT', 'GT', 'GG'], 'rs8175347':['(TA)6/(TA)6', '(TA)6/(TA)7', '(TA)7/(TA)7'], 'rs716274':['AA', 'AG', 'GG'], 'rs3212986':['AA', 'CA', 'CC'], 'rs1045642':['AA', 'AG', 'GG'], 'rs4353229':['TT', 'CC', 'TC']}


dic_500gene = {'rs817345':['(TA)6/(TA)6', '(TA)6/(TA)7', '(TA)7/(TA)7'], 'rs25487':['AA', 'AG','GG'], 'rs1801133':['AA', 'GA', 'GG'],
'rs11615':['TT', 'CT', 'CC'], '1695':['AA', 'AG', 'GG'], 'rs67376798':['AA', 'TA', 'TT'],  'rs1801019':['GG', 'GC', 'CC'], 'rs662':['TT', 'CT','CC'], 'rs1800566':['AA', 'GA', 'GG'], 'rs716274':['AA', 'AG', 'GG'], 'rs4353229':['TT', 'TC', 'CC'], 'rs1042522':['GG', 'CG', 'CC'], 'rs1045642':['AA', 'AG', 'GG'], 'rs3212986':['AA', 'CA', 'CC'], 'rs4148323':['AA', 'GA', 'GG'], 'rs1517114':['GG', 'CG', 'GG'], 'rs4880':['AA','AG', 'GG'], 'rs1056892':['AA', 'GA', 'GG'], 'rs11545078':['TT', 'CT', 'CC'], 'rs9937':['AA', 'AG', 'GG'], 'rs2072671':['AA', 'AC', 'CC'], 'rs3918290':['TT', 'CT', 'CC'], 'rs2740574':['TT','CT', 'CC'], 'rs2228001':['TT', 'GT', 'GG'] }

dic_aitong = {'rs1799971':['AA', 'AG', 'GG'], 'rs4680':['AA', 'AG', 'GG'], 'rs1045642':['GG', 'AG', 'GG'], 'rs1057910':['AA', 'AC', 'CC'],'rs20417':['CC', 'GC', 'CC']}
dic_aitong500 = {'rs1799971':['AA', 'AG', 'GG'], 'rs4680':['AA', 'AG', 'GG'], 'rs1045642':['GG', 'AG', 'GG'], 'rs1057910':['AA', 'AC', 'CC'],'rs20417':['CC', 'GC', 'CC'], 'rs12948783':['AA', 'AG', 'GG']}
###################
for name in project_dic.keys():
    if '肺癌37基因（靶向+化疗+癌痛）' in project_dic[name].split('--')[0]:
        gene_37[name] = correctGT(mypath+'/'+project_dic[name].split('--')[1], dic_37gene)
        gene_aitong[name] = correctGT(mypath+'/'+project_dic[name].split('--')[1], dic_aitong)
    elif '实体瘤泛癌种50基因（靶向+化疗+癌痛）' in project_dic[name].split('--')[0]:
        gene_50[name] = correctGT(mypath+'/'+project_dic[name].split('--')[1], dic_50gene)
        gene_aitong[name] = correctGT(mypath+'/'+project_dic[name].split('--')[1], dic_aitong)
    elif '实体瘤泛癌种500基因' in project_dic[name].split('--')[0]:
        gene_500[name] = correctGT(mypath+'/'+project_dic[name].split('--')[1], dic_500gene)
        gene_aitong500[name] = correctGT(mypath+'/'+project_dic[name].split('--')[1], dic_aitong500)
    elif '肺癌21基因（靶向+癌痛）' in project_dic[name].split('--')[0]:
        gene_aitong[name] = correctGT(mypath+'/'+project_dic[name].split('--')[1], dic_aitong)

    




In [15]:
gene_aitong


{'谭兴英': ['rs1799971--AA',
  'rs4680--GG',
  'rs1045642--GG',
  'rs1057910--AA',
  'rs20417--CC'],
 '刘淑玲': ['rs1799971--AA',
  'rs4680--GG',
  'rs1045642--(TA)7/(TA)7',
  'rs1057910--AA',
  'rs20417--CC'],
 '郭元海': ['rs1799971--AG',
  'rs4680--AG',
  'rs1045642--GG',
  'rs1057910--AA',
  'rs20417--GC'],
 '曹喜成': ['rs1799971--GG',
  'rs4680--AG',
  'rs1045642--GG',
  'rs1057910--AA',
  'rs20417--CC']}

In [16]:
"""
wb = openpyxl.load_workbook('/Users/andy/Desktop/化疗药/泛癌种500基因化疗/实体瘤500基因.xlsx')
sheet = wb['Sheet1']
a = sheet.max_row
index = 0
sheet['F'+str(a+1)] = '黄金玉'
for i in sheet['K'+str(a+1):'AH'+str(a+1)][0]:
    i.value = gene_500['孙良君'][index].split('--')[1]
    index += 1

wb.save('/Users/andy/Desktop/化疗药/泛癌种500基因化疗/实体瘤500基因.xlsx')
"""

"\nwb = openpyxl.load_workbook('/Users/andy/Desktop/化疗药/泛癌种500基因化疗/实体瘤500基因.xlsx')\nsheet = wb['Sheet1']\na = sheet.max_row\nindex = 0\nsheet['F'+str(a+1)] = '黄金玉'\nfor i in sheet['K'+str(a+1):'AH'+str(a+1)][0]:\n    i.value = gene_500['孙良君'][index].split('--')[1]\n    index += 1\n\nwb.save('/Users/andy/Desktop/化疗药/泛癌种500基因化疗/实体瘤500基因.xlsx')\n"

In [17]:
def fun_write(file_name, name_index, rs_index): #将校正过的数据写入相应输入文件:file_name:基因型结果输入Excel文件，name_index:Excel姓名坐标，rs_index:rs位点坐标
    wb = openpyxl.load_workbook(file_name)
    sheet = wb['Sheet1']
    a = sheet.max_row
    index = 0
    

In [18]:
dic_inputfile = {'gene_37':['/Users/andy/Desktop/化疗药/肺癌化疗37基因/肺癌化疗检测报告20191014(1).xlsx', 'F', 'K', 'V',gene_37], 'gene_50':['/Users/andy/Desktop/化疗药/泛癌种50基因/泛癌50基因20191015.xlsx', 'F', 'K', 'AD', gene_50], 'gene_500':['/Users/andy/Desktop/化疗药/泛癌种500基因化疗/实体瘤500基因.xlsx', 'F', 'K', 'AH', gene_500], 'gene_aitong':['/Users/andy/Desktop/化疗药/癌痛/圣庭癌痛报告结果(1).xlsx', 'C', 'J', 'N', gene_aitong], 'gene_aitong500':['/Users/andy/Desktop/化疗药/癌痛/圣庭500基因癌痛报告结果.xlsx', 'C','J','O', gene_aitong500]}

for i in ['gene_37', 'gene_50', 'gene_500', 'gene_aitong', 'gene_aitong500']:
    if dic_inputfile[i][4]:
        
        for name in dic_inputfile[i][4].keys():
            wb = openpyxl.load_workbook(dic_inputfile[i][0])
            sheet = wb['Sheet1']
            a = sheet.max_row
            index = 0
            sheet[dic_inputfile[i][1]+str(a+1)] = name
            for cell in sheet[dic_inputfile[i][2]+str(a+1):dic_inputfile[i][3]+str(a+1)][0]:
                cell.value = dic_inputfile[i][4][name][index].split('--')[1]
                index += 1
            
            wb.save(dic_inputfile[i][0])
          
